# DESaster Simulation Set Up Template

## Required Modules

In [3]:
import sys, random
desaster_path = "/Users/geomando/Dropbox/github/SeaGrantSimulation"
sys.path.append(desaster_path)
import simpy
from simpy import Resource, Container, FilterStore
from simpy.util import start_delayed
import pandas as pd
import numpy as np
from desaster import entities, capitals, request, io, movement, search, rebuild

## Input Data

In [4]:
scenario_file = '../inputs/scenario_test_renters.xlsx'

In [5]:
# Create Pandas dataframe of attribute data for all owners to be modeled in the simulation
owners_df = pd.read_excel(scenario_file, sheetname='owners')
owners_df

,Name,Income,Savings,Insurance,Address,Occupancy,Cost,Bedrooms,Bathrooms,Area,Year Built,Value,Damage State,Tenure Pref,Tenure,Occupancy Pref
0,Alfred,30000,5000,0.00,62 That St,Mobile Home,1500,1,1,1100,1920,100000,Complete,Own,Own,Single Family Dwelling
1,Bruce,100000,1000000,0.85,720 This Rd,Single Family Dwelling,3500,4,5,5000,1920,10000000,Moderate,Own,Own,Single Family Dwelling
2,Selena,10000,2500,0.00,1001 Other Ave,Single Family Dwelling,1000,2,1,1200,1960,10000,Extensive,Own,Own,Single Family Dwelling
3,Fish,50000,1000,0.85,26000 Out There Lane,Single Family Dwelling,3000,3,2,2000,2010,800000,Extensive,Own,Own,Single Family Dwelling


In [6]:
# Create Pandas dataframe of attribute data for all renters to be modeled in the simulation
renters_df = pd.read_excel(scenario_file, sheetname='renters')
renters_df

,Name,Income,Savings,Insurance,Address,Occupancy,Cost,Bedrooms,Bathrooms,Area,Year Built,Value,Damage State,Tenure Pref,Tenure,Occupancy Pref
0,Ivy,30000,10000,0,262 That St,Mobile Home,1000,1,1,700,1920,100000,Extensive,Rent,Rent,Single Family Dwelling
1,Edward,100000,50000,10000000,4720 This Rd,Single Family Dwelling,2500,3,2,5000,1920,10000000,Complete,Own,Rent,Single Family Dwelling
2,Oswald,10000,100,0,2301 Other Ave,Multi Family Dwelling,750,0,1,250,1960,10000,Complete,Rent,Rent,Multi Family Dwelling
3,James,50000,1000,550000,74000 Out There Lane,Single Family Dwelling,2000,2,2,2000,2010,800000,Complete,Own,Rent,Single Family Dwelling


In [7]:
# Set input data for all human capital types, as dict or Pandas Series
# .loc stuff is to convert the DataFrame to a Series ... data will function the same as a dictionary as well
human_cap_data = pd.read_excel(scenario_file, sheetname='human_capital', index_col=0).iloc[:,0]
human_cap_data

Type
Contractors             2
Engineers               2
FEMA Processors         1
Inspectors              4
Insurance Adjusters    40
Loan Processors        40
Permit Processors       2
Name: Quantity, dtype: int64

In [8]:
human_cap_data['Contractors'] = 4
human_cap_data['Engineers'] = 4
human_cap_data['FEMA Processors'] = 4
human_cap_data['Inspectors'] = 12
human_cap_data['Insurance Adjusters'] = 4
human_cap_data['Loan Processors'] = 4
human_cap_data['Permit Processors'] = 4

__Set input data for all financial capital types, as dict or Pandas Series.__

In [9]:
financial_cap_data = pd.read_excel(scenario_file, sheetname='financial_capital', index_col=0).iloc[:,0]
financial_cap_data

Type
Building Materials     2000000
FEMA Aid              35000000
Name: Quantity, dtype: int64

In [10]:
financial_cap_data['Building Materials'] = 2000000
financial_cap_data['FEMA Aid'] = 2000000

__Create Pandas dataframe of attribute data for all vacant homes for sale to be modeled in the simulation.__

In [11]:
forsale_stock_df = pd.read_excel(scenario_file, sheetname='forsale_stock')
forsale_stock_df

,Address,Occupancy,Cost,Bedrooms,Bathrooms,Area,Year Built,Value,Damage State
0,100 New Ave,Mobile Home,100,1,1,700,1920,99999,None
1,101 New Ave,Single Family Dwelling,100000,6,5,5000,1920,9999,None
2,102 New Ave,Multi Family Dwelling,10,0,1,250,1960,9999,None
3,103 New Ave,Single Family Dwelling,2000,4,2,2000,2010,800000,None


__Create Pandas dataframe of attribute data for all vacant rentals to be modeled in the simulation.__

In [12]:
forrent_stock_df = pd.read_excel(scenario_file, sheetname='forrent_stock')
forrent_stock_df

,Address,Occupancy,Cost,Bedrooms,Bathrooms,Area,Year Built,Value,Damage State
0,100 Old Ave,Mobile Home,100,1,1,700,1920,99999,Complete
1,101 Old Ave,Single Family Dwelling,100000,2,2,5000,1920,9999,Complete
2,102 Old Ave,Multi Family Dwelling,10,0,1,250,1960,9999,Complete
3,103 Old Ave,Single Family Dwelling,2000,3,2,2000,2010,800000,Complete


## Simulation Initiation

__Set simulation environment__

In [13]:
simulation = simpy.Environment()

__Indicate whether want to keep track of the stories of each entity (household) in the simulation.__

In [14]:
write_story = True

__Populate the simulation with human and financial capital data.__

In [15]:
financial_capital = capitals.FinancialCapital(simulation, financial_cap_data) 

human_capital = capitals.HumanCapital(simulation, human_cap_data)

__Create and populate a FilterStore as a vacant housing stock.__

In [16]:
forsale_stock = capitals.importHousingStock(simulation, forsale_stock_df)

__Create and populate a FilterStore as a vacant rental stock.__

In [17]:
forrent_stock = capitals.importHousingStock(simulation, forrent_stock_df)

__Create an empty FilterStore to use as the occupied housing stock.__

In [18]:
owned_stock = FilterStore(simulation)
rented_stock = FilterStore(simulation)

__Import data on owner occupiers. Everytime a household's residence is created put it in the occupied housing stock.__

In [19]:
owners = entities.importOwners(simulation, owned_stock, owners_df, write_story)

__Import data on renters. Everytime a renter's residence is created put it in the occupied rental stock.__

In [20]:
renters = entities.importRenters(simulation, rented_stock, renters_df, write_story)

__Assign landlords (with randomly generated names) to renters.__

In [21]:
landlords = []

for renter in renters:
    landlords.append(entities.Landlord())
    
entities.assignLandlords(renters, landlords, write_story)

In [22]:
renters[1].residence == renters[1].landlord.residence

True

__Write a master process for landlords that combines process and functions from search, rebuild, and request modules.__

In [23]:
def landlord_process(simulation, human_capital, financial_capital, entity, write_story):
        
    yield simulation.process(request.inspection(simulation, human_capital, entity.residence, entity, write_story))
    
    # Specify the event sequence for households from the time of the hazard through the decisions to relocate 
    # or rebuild
    if entity.residence.damage_state != 'None':
        
        money_patience = 365  # days until give up the search for rebuild money

        # Search for rebuild money
        yield simulation.process(search.rebuild_money(simulation, human_capital, 
                                                        financial_capital, entity, 
                                                        money_patience, write_story))
        
        if entity.gave_up_money_search == True:
            entity.residence = []
            entity.tenant.prior_residence = entity.tenant.residence
            entity.tenant.residence = []
            return
        
        # If home is completely damaged, evict tenant
        if entity.residence.damage_state == 'Complete':
            entity.residence = []
            entity.tenant.prior_residence = entity.tenant.residence
            entity.tenant.residence = []
            
            if write_story == True:
                entity.tenant.story.append(
                '{0} was permanently evicted because the {1} was demolished. '.format(
                entity.tenant.name, entity.tenant.prior_residence.occupancy.lower())
                )
            return

        if entity.residence.damage_state != 'None':
            yield simulation.process(request.engineering_assessment(simulation, human_capital, entity, write_story))

            yield simulation.process(request.permit(simulation, human_capital, entity, write_story))

            yield simulation.process(rebuild.home(simulation, human_capital, financial_capital, entity, write_story))
            
            vacancy_level = len(forrent_stock.items) - len(forrent_stock.get_queue)
            vacancy_rate = vacancy_level / len(forrent_stock.items)
            rent_delta = 9.08 - 0.21 * vacancy_rate
              
            print(len(forrent_stock.items))
            print(len(forrent_stock.get_queue))
            print(vacancy_level)
            print(vacancy_rate)
            print(rent_delta)

__Write a master process for owner-occupiers that combines process and functions from search, rebuild, and request modules.__

In [24]:
def owner_process(simulation, human_capital, financial_capital, entity, write_story):
        
    yield simulation.process(request.inspection(simulation, human_capital, entity.residence, entity, write_story))
    
    # Specify the event sequence for households from the time of the hazard through the decisions to relocate 
    # or rebuild
    if entity.residence.damage_state != 'None':
        
        money_patience = 365  # days until give up the search for rebuild money

        # Search for rebuild money
        yield simulation.process(search.rebuild_money(simulation, human_capital, 
                                                        financial_capital, entity, 
                                                        money_patience, write_story))
        
        if entity.gave_up_money_search == True:
                return
        
        # If home is completely damaged, search for a new home to purchase.
        if entity.residence.damage_state == 'Complete':
            
            home_patience = 550  # days until give up the search for a new home

            search_outcome = yield simulation.process(search.permanent_housing(simulation, entity, home_patience, forsale_stock, human_capital, write_story))

            if entity.gave_up_home_search == True:
                return

        if entity.residence.damage_state != 'None':
            yield simulation.process(request.engineering_assessment(simulation, human_capital, entity, write_story))

            yield simulation.process(request.permit(simulation, human_capital, entity, write_story))

            yield simulation.process(rebuild.home(simulation, human_capital, financial_capital, entity, write_story))
            
            yield simulation.process(request.reoccupy(simulation, entity, write_story))
               

__Write a master process for renters.__

In [25]:
def renter_process(simulation, human_capital, financial_capital, entity, write_story):
        
    yield simulation.process(landlord_process(simulation, human_capital, financial_capital, entity.landlord, write_story))

    if entity.residence:
        yield simulation.process(request.reoccupy(simulation, entity, write_story))
    
    if not entity.residence:
        search_patience = 550  # days until give up the search for a new home
        search_outcome = yield simulation.process(search.permanent_housing(simulation, entity, search_patience, forrent_stock, human_capital, write_story))

__Initiate the master process for each owner to be modeled in the simulation.__

In [26]:
for i in range(len(owners)):
    simulation.process(owner_process(simulation, human_capital, financial_capital, owners[i], write_story))

__Initiate the master process for each renter to be modeled in the simulation.__

In [27]:

for i in range(len(renters)):
    simulation.process(renter_process(simulation, human_capital, financial_capital, renters[i], write_story))

__Do inspections on all of the for sale stock.__
__Schedule an event that randomly fixes moderately or completely damaged homes in the vacant housing stock with probability = fix_probability__

In [28]:
for home in forsale_stock.items:
    simulation.process(request.inspection(simulation, human_capital, home))

fix_probability = 1.0
fix_schedule = 100

start_delayed(simulation, rebuild.stock(simulation, forsale_stock, fix_probability), fix_schedule)

<Process(starter) object at 0x10d723048>

In [29]:
# # Do inspections on all of the rental stock
# for home in forrent_stock.items:
#     simulation.process(request.inspection(simulation, human_capital, home))

# # Schedule an event that randomly fixes moderately or completely damaged homes in the vacant housing stock
# # with probability = fix_probability
# fix_probability = 0.0
# fix_schedule = 200

# start_delayed(simulation, rebuild.stock(simulation, forrent_stock, fix_probability), fix_schedule)

In [30]:
simulation.run()

1 homes in the vacant building stock were fixed on day 100.
4
3
1
0.25
9.0275


## Outputs

__Owner summary statistics__

In [31]:
num_damaged = 0
num_rebuilt = 0
num_gave_up_money_search = 0
num_relocated = 0
num_homesearch = 0
num_gave_up_home_search = 0

for household in owners:
    if household.money_search_start != None: num_damaged += 1
    if household.home_get != None: num_rebuilt += 1
    if household.gave_up_money_search: num_gave_up_money_search += 1
    if household.home_search_start != None: num_homesearch += 1
    if household.home_search_stop != None: num_relocated += 1
    if household.gave_up_home_search: num_gave_up_home_search += 1
        
print('{0} out of {1} owners suffered damage to their homes.\n'.format(num_damaged, len(owners)),
      '{0} out of {1} owners rebuilt or repaired their damaged home.\n'.format(num_rebuilt, len(owners)),
        '{0} out of {1} owners gave up searching for money.\n'.format(num_gave_up_money_search, len(owners)),
      '{0} out of {1} owners searched for a new home.\n'.format(num_homesearch, len(owners)),
        '{0} out of {1} owners bought a new home.\n'.format(num_relocated, len(owners)),
        '{0} out of {1} owners gave up searching for a home.'.format(num_gave_up_home_search, len(owners))
      )

4 out of 4 owners suffered damage to their homes.
 3 out of 4 owners rebuilt or repaired their damaged home.
 0 out of 4 owners gave up searching for money.
 1 out of 4 owners searched for a new home.
 1 out of 4 owners bought a new home.
 0 out of 4 owners gave up searching for a home.


__Owner stories__

In [32]:
owners[0].story

['Alfred owns and lives in a 1 bedroom mobile home at 62 That St worth $100,000. ',
 "Alfred's mobile home was inspected 10 days after the event and suffered $100,000 of damage.",
 'Alfred submitted a request to FEMA 10 days after the event. ',
 'Alfred received $30,000 from FEMA 30 days after the event. ',
 'Alfred submitted a loan application 30 days after the event. ',
 'Alfred received a loan for $70,000 60 days after the event. ',
 'It took Alfred 50 days to receive enough financial assistance and now has $105,000 to rebuild. ',
 'Alfred started searching for a new mobile home 60 days after the event. ',
 'On day 60, Alfred purchased a mobile home at 100 New Ave with a value of $99,999 and $0 of damage. ']

In [33]:
owners[1].story

['Bruce owns and lives in a 4 bedroom single family dwelling at 720 This Rd worth $10,000,000. ',
 "Bruce's single family dwelling was inspected 10 days after the event and suffered $1,000,000 of damage.",
 'Bruce submitted an insurance claim 10 days after the event. ',
 "Bruce's insurance deductible is greater than the value of damage. ",
 'It took Bruce 0 days to receive enough financial assistance and now has $1,000,000 to rebuild. ',
 'Bruce received an engineering assessment 35 days after the event. ',
 'Bruce received permit approval 70 days after the event. ',
 "Bruce's single family dwelling was repaired 100 days after the event, taking 30 days to repair. ",
 'There were insufficient construction materials available in the area for Bruce to rebuild. ',
 'Bruce reoccupied the single family dwelling 102 days after the event. ']

In [34]:
''.join(owners[2].story)

"Selena owns and lives in a 2 bedroom single family dwelling at 1001 Other Ave worth $10,000. Selena's single family dwelling was inspected 10 days after the event and suffered $4,470 of damage.Selena submitted a request to FEMA 10 days after the event. Selena received $4,470 from FEMA 30 days after the event. It took Selena 20 days to receive enough financial assistance and now has $6,970 to rebuild. Selena received an engineering assessment 55 days after the event. Selena received permit approval 90 days after the event. Selena's single family dwelling was repaired 180 days after the event, taking 90 days to repair. Selena reoccupied the single family dwelling 182 days after the event. "

In [35]:
owners[3].story

['Fish owns and lives in a 3 bedroom single family dwelling at 26000 Out There Lane worth $800,000. ',
 "Fish's single family dwelling was inspected 10 days after the event and suffered $357,600 of damage.",
 'Fish submitted an insurance claim 10 days after the event. ',
 'Fish received a $237,600 insurance payout 25 days after the event. ',
 'Fish submitted a request to FEMA 25 days after the event. ',
 'Fish received $30,000 from FEMA 45 days after the event. ',
 'Fish submitted a loan application 45 days after the event. ',
 'Fish received a loan for $90,000 75 days after the event. ',
 'It took Fish 65 days to receive enough financial assistance and now has $358,600 to rebuild. ',
 'Fish received an engineering assessment 100 days after the event. ',
 'Fish received permit approval 135 days after the event. ',
 "Fish's single family dwelling was repaired 225 days after the event, taking 90 days to repair. ",
 'Fish reoccupied the single family dwelling 227 days after the event. ']

__Renter summary statistics__

In [36]:
num_damaged = 0
num_rebuilt = 0
num_gave_up_money_search = 0
num_relocated = 0
num_displaced = 0
num_gave_up_home_search = 0

for renter in renters:

    if renter.landlord.money_search_start > 0.0: num_damaged += 1
    if renter.landlord.home_get > 0.0: num_rebuilt += 1
    if renter.landlord.gave_up_money_search: num_gave_up_money_search += 1
    if not renter.residence: num_displaced += 1
        
print('{0} out of {1} renters\' homes suffered damage.\n'.format(num_damaged, len(renters)),
      '{0} out of {1} renters\' damaged home was rebuilt or repaired.\n'.format(num_rebuilt, len(renters)),
      '{0} out of {1} renters\' were displaced.\n'.format(num_displaced, len(renters)),
      )

TypeError: unorderable types: NoneType() > float()

__Renters Stories__

In [ ]:
renters[0].story + renters[0].landlord.story

In [ ]:
renters[1].story + renters[1].landlord.story

In [ ]:
renters[1].story + renters[2].landlord.story

In [ ]:
renters[2].story + renters[3].landlord.story

In [ ]:
if not renters[1].residence: print('Im homeless.')

In [ ]:
isinstance(renters[1], entities.Renter)

In [ ]:
renters[2].residence

In [ ]:
#fills the empty dataframe we made above for the output. incredibly badly written
a = list(vars(owners[3]).keys()) #gets all potential column names
# a.remove("household");a.remove("residence") #remove the stuff we don't want
# a.append("latitude");a.append("longitude") #add stuff we do want
df = pd.DataFrame(columns=a)
iters = 0
att_itter = 0
new_column={}
log = []
for i in owners: #loop through all entities
#     i.latitude = i.owner["Latitude"] #extracting lat and long from the residence object
#     i.longitude = i.owner["Longitude"]
    for att in a: #loop through the attributes in our list of column names we want
        try:
            new_column[att] = i.__getattribute__(att) #set the b dictionary
            #mydata[att]= i.__getattribute__(att)
            
        except ValueError:
            new_column[att] = np.nan
        except AttributeError as e:
            new_column[att] = np.nan
            log.append("Household {0} had an attr error, {1}".format(i.name, e))
        finally:
            att_itter += 1
    mydata=pd.DataFrame([new_column]) #this turns our newly made column into a database where it can be combined with the df

    df = df.append(mydata, ignore_index=True)

    iters += 1
    

print(iters)
print(att_itter)



In [ ]:
df.head()
output_path = "../outputs/output_df.csv"
df.to_csv(output_path)

In [ ]:
df